## Online Lab: Pandas with Filtering II

In this online lab, we will use the NFL data set. The data is at "data\football_data".

We are going to review the following knowledge points:

1. Groupby with Transform
2. Groupby with Filtering
3. Merges

#### Ex1:  Read in the games and teams dataset

In [3]:
import pandas as pd

df_games = pd.read_csv("data/football_data/nflgames.csv")
df_teams = pd.read_csv("data/football_data/nflteams.csv")
df_teams.head()

,TeamID,TeamName,TeamCapsAbrv,TeamAbrv
0,1,Baltimore Ravens,RAV,rav
1,2,Denver Broncos,DEN,den
2,3,Oakland Raiders,RAI,rai
3,4,Philadelphia Eagles,PHI,phi
4,5,Dallas Cowboys,DAL,dal


#### Ex2:  Compute the average homescores for all teams in this dataset

In [5]:
og_avg_homescores = df_games["HomeScore"].mean()
og_avg_homescores

24.973958333333332

#### Ex3:  Create a categorical variable called temp_cate, which is equal to high if temp > 80, middle if temp is in [65, 80] and low if temp < 65.

In [8]:
def temp_cate (temp):
    if temp > 80:
        return "high"
    elif temp < 65: 
        return "low"
    else:
        return "middle"
    
df_games["temp_cate"] = df_games["Temp"].apply(temp_cate)
df_games.head()

,GameID,Week,HomeTeamID,AwayTeamID,HomeScore,AwayScore,DayOfWeek,TimeOfDay,FieldType,Temp,Wind,temp_cate
0,1,1,1,29,16,23,Sun,Day,sportturf,74,8,middle
1,2,2,1,28,26,6,Thu,Night,sportturf,82,6,high
2,3,3,27,1,21,23,Sun,Day,grass,71,23,middle
3,4,4,1,15,38,10,Sun,Day,sportturf,78,4,middle
4,5,5,7,1,20,13,Sun,Day,fieldturf,0,0,low


#### Ex4:  Standardize A team's homescore based on the average and standard deviations of homescores of all games in that temperature region.

In [14]:
df_games["standardized_homescore"] = df_games.groupby(by = "temp_cate")["HomeScore"]\
                                    .transform(lambda x: (x - x.mean()) / x.std())
df_games.head()


,GameID,Week,HomeTeamID,AwayTeamID,HomeScore,AwayScore,DayOfWeek,TimeOfDay,FieldType,Temp,Wind,temp_cate,standardized_homescore
0,1,1,1,29,16,23,Sun,Day,sportturf,74,8,middle,-0.672074
1,2,2,1,28,26,6,Thu,Night,sportturf,82,6,high,0.094846
2,3,3,27,1,21,23,Sun,Day,grass,71,23,middle,-0.160919
3,4,4,1,15,38,10,Sun,Day,sportturf,78,4,middle,1.577007
4,5,5,7,1,20,13,Sun,Day,fieldturf,0,0,low,-0.522104


#### Ex5: Filter out the teams whose average standardized homescores are below 0

In [16]:
df_games2 = df_games.groupby(by = "HomeTeamID").filter(lambda  x: x["standardized_homescore"].mean() >= 0)
df_games2.shape

(84, 13)

#### Ex6:  Compute the average (non-standardized) homescores for all teams in this new dataset

In [18]:
avg_homescores = df_games2["HomeScore"].mean()
avg_homescores, og_avg_homescores

(30.666666666666668, 24.973958333333332)

#### Ex7:  Create a dataset contains 2 columns: one is the team's average (non-standardized) homescores for the non-filtered teams, and the other is the team's abbreviation

In [21]:
df_team2 = df_games2.groupby("HomeTeamID")\
            .apply(lambda df: pd.Series({"avg_HomeScore" : df["HomeScore"].mean()}))
df_team2 = df_team2.reset_index()
df_team2 = df_team2.merge(df_teams, how = "left", left_on = "HomeTeamID", right_on = "TeamID")
df_team2 = df_team2[["TeamAbrv", "avg_HomeScore"]]
df_team2

,TeamAbrv,avg_HomeScore
0,rav,27.166667
1,den,35.333333
2,phi,36.666667
3,clt,29.571429
4,sdg,25.666667
5,kan,24.833333
6,nwe,35.666667
7,atl,30.000000
8,nor,27.000000
9,sea,28.666667
